__Calculating rotation on a single seed__

In [22]:
import pandas as pd
import numpy as np
import py3Dmol
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm
import os
import pickle 

def get_box(filename):
    positions, forces, masses = [], [], []
    with open(filename, 'r') as f:
        for _ in range(5):
            next(f)
        box_size_line = f.readline().strip()
        box_size = np.array(list(map(float, box_size_line.split())))[1]
    return box_size

def load_initial_indices(filename, center, sph_rad):
    '''
    Input
    1) filename: what file do you want to look at (Code written exclusively for LAMMPS format output file)
    2) sph_rad: radius of the sample sphere
    Output
    1) indices: Particle indices of the atoms in the seeds before the SCG 
    '''
    indices, positions = [], []
    with open(filename, 'r') as f:
        for _ in range(3):
            next(f)
        num_atoms = int(f.readline().strip())
        next(f)
        box_size_line = f.readline().strip()
        box_size = float(box_size_line.split()[1])
        for _ in range(3):
            next(f)
        data = []
        for _ in range(num_atoms):
            data.append(f.readline().split())
        # Create DataFrame
        df = pd.DataFrame(data, columns=["Atoms_id", "type", "x",  "y", "z", "Q6_1", "Q6_2", "Q6_3", "Q6_4", "Nc_6", "Nc_8","str_x","str_y","str_z"])
        df = df.astype({"Atoms_id": int, "type":int, "x": float, "y": float, "z": float})

        # Filter atoms within the sphere
        distances = np.sqrt((df["x"] - center[0])**2 + (df["y"] - center[1])**2 +(df["z"] - center[2])**2)
        df_filtered = df[distances <= sph_rad].copy()
        df_filtered = df_filtered.sort_values(by="Atoms_id")
        indices = df_filtered[["Atoms_id"]].values
        positions = df_filtered[["x", "y", "z"]].values
    return indices, positions

def load_snapshots(filename, indices):
    '''
    Input
    1) filename: what file do you want to look at (Code written exclusively for LAMMPS format output file)
    2) sph_rad: radius of the sample sphere
    Output
    1) snapshots: dict of particle positions for atoms with Atoms_id = indices
    '''
    snapshots = {}
    with open(filename, 'r') as f:
        snapshot_index = 0
        while True:
            # Read and process the header for each snapshot
            header = [f.readline().strip() for _ in range(9)]
            if not header[0]:  # End of file
                break
            num_atoms = int(header[3])
            # Read the atom data for the snapshot
            data = []
            for _ in range(num_atoms):
                line = f.readline()
                if not line:
                    break
                data.append(line.split())
            df = pd.DataFrame(data, columns=["Atoms_id", "type", "x", "y", "z", "Q6_1", "Q6_2", "Q6_3", "Q6_4", "Nc_6", "Nc_8", "str_x", "str_y", "str_z"])
            df = df.astype({"Atoms_id": int, "type": int, "x": float, "y": float, "z": float})
            df = df.sort_values(by="Atoms_id")
            df_filtered = df[df['Atoms_id'].isin(indices)]
            # print(f"Running on snapshot {snapshot_index+1} and has {len(df_filtered)} atoms selected.")
            # Store indices and positions for the snapshot
            indices = df_filtered["Atoms_id"].values.flatten()
            positions = df_filtered[["x", "y", "z"]].values
            # snapshots[snapshot_index] = {"indices": indices,"positions": positions}
            snapshots[snapshot_index] = positions
            snapshot_index += 1

    return snapshots

def compute_quaternion_rotation(ref_positions, current_positions):
    """
    Compute the quaternion representing the rotation between two sets of points.

    Parameters:
    ref_positions (ndarray): Reference positions of key atoms (N x 3).
    current_positions (ndarray): Current positions of the same key atoms (N x 3).

    Returns:
    quaternion (ndarray): Quaternion representing the rotation (4,).
    """
    # Center both sets of positions to remove translational components
    ref_centered = ref_positions - np.mean(ref_positions, axis=0)
    current_centered = current_positions - np.mean(current_positions, axis=0)

    # Compute the covariance matrix
    covariance_matrix = np.dot(current_centered.T, ref_centered)

    # Perform Singular Value Decomposition (SVD)
    U, _, Vt = np.linalg.svd(covariance_matrix)

    # Compute the rotation matrix
    rotation_matrix = np.dot(U, Vt)

    # Ensure a proper rotation (det(rotation_matrix) = 1)
    if np.linalg.det(rotation_matrix) < 0:
        U[:, -1] *= -1
        rotation_matrix = np.dot(U, Vt)

    # Convert the rotation matrix to a quaternion
    quaternion = R.from_matrix(rotation_matrix).as_quat()

    return quaternion

def main():
    filename = 'combined_Nc.dump'
    box = (get_box(filename))
    print(f"Box Size: {box}")
    seed_center = [0.5*box-16.0, 0.5*box+16.0, 0.5*box-16.0]
    seed_indices, ref_coords = load_initial_indices(filename, seed_center, sph_rad=15.0)
    seed_indices = seed_indices.flatten().tolist()
    # Define the path where the snapshot dict will be saved
    snapshot_file = 'snapshots.pkl'

    # Check if the snapshot dict file exists
    if os.path.exists(snapshot_file):
        print(f"Loading snapshots from {snapshot_file}")
        with open(snapshot_file, 'rb') as f:
            snapshots = pickle.load(f)
    else:
        print("Snapshot file not found, calculating snapshots.")
        snapshots = load_snapshots(filename, seed_indices)

        # Save the snapshot dict for future use
        with open(snapshot_file, 'wb') as f:
            pickle.dump(snapshots, f)
        print(f"Snapshots saved to {snapshot_file}")
    
    rot_angle = []
    print("Now running rotation calculation.")
    # for snap_idx, snap_data in snapshots.items():
    for snap_idx, snap_data in tqdm(snapshots.items(), desc="Processing Snapshots", total=len(snapshots)):
        rot = (compute_quaternion_rotation(ref_coords, snap_data))
        print(np.arccos(rot[0])*2)
        rot_angle.append(compute_quaternion_rotation(ref_coords, snap_data))
if __name__ == '__main__':
    main()

Box Size: 181.7376018642
Loading snapshots from snapshots.pkl
Now running rotation calculation.


Processing Snapshots: 100%|██████████| 60/60 [00:00<00:00, 2112.79it/s]

3.1415926535897936
3.149647636435305
3.123130663396301
3.100850390515788
3.0895955746199264
3.088208224009359
3.1009709018930653
3.092340610179686
3.058050162611333
3.074044826464007
3.1175062325676866
3.0416668376272407
3.08628200381835
3.03483162349295
3.0225053877638346
2.925052553505601
2.941112991039825
3.0557867392423312
3.0214485160414064
3.061562641447426
3.0988881046383243
3.0997093506784563
3.0193876468489598
3.0623584796993004
3.0607007631514334
3.041163078771797
2.934391684988949
3.014896227529763
3.0590904910319643
3.1315568705411896
3.1629375816431655
3.214549758891139
3.243246029900034
3.1737953930656695
3.182020041790782
3.1540048231875017
3.1510935191626586
3.225261157448519
3.1602138852839734
3.1142753849523275
3.1801097687938733
3.2257917173627955
3.195002497705309
3.1451293101170963
3.1928389096649594
3.1494742112290135
3.2251893193855876
3.1617402309027423
3.146064779497329
3.193850133707201
3.1401702619194993
3.168271606754367
3.200122332648284
3.126409519514478
3

__Calculating angle between two seeds__

In [21]:
def main():
    filename = 'combined_Nc.dump'
    box = (get_box(filename))
    print(f"Box Size: {box}")
    seed_center_I = [0.5*box-16.0, 0.5*box+16.0, 0.5*box-16.0]
    seed_indices_I, ref_coords_I = load_initial_indices(filename, seed_center_I, sph_rad=15.0)
    seed_indices_I = seed_indices_I.flatten().tolist()
    
    seed_center_II = [0.5*box+16.0, 0.5*box-15.0, 0.5*box+16.0]
    seed_indices_II, ref_coords_II = load_initial_indices(filename, seed_center_II, sph_rad=15.0)
    seed_indices_II = seed_indices_II.flatten().tolist()

    if len(seed_indices_I) > len(seed_indices_II):
        seed_indices_I = seed_indices_I[:len(seed_indices_II)]
    else:
        seed_indices_II = seed_indices_I[:len(seed_indices_I)]
        
    snapshot_file_I = 'snapshots_I.pkl'
    snapshot_file_II = 'snapshots_II.pkl'

    # Check if the snapshot dict file exists
    if os.path.exists(snapshot_file_I):
        print(f"Loading snapshots from {snapshot_file_I}")
        with open(snapshot_file_I, 'rb') as f:
            snapshots_I = pickle.load(f)
    else:
        print("Snapshot file not found, calculating snapshots.")
        snapshots = load_snapshots(filename, seed_indices_I)

        # Save the snapshot dict for future use
        with open(snapshot_file_I, 'wb') as f:
            pickle.dump(snapshots, f)
        print(f"Snapshots saved to {snapshot_file_I}")

    if os.path.exists(snapshot_file_II):
        print(f"Loading snapshots from {snapshot_file_II}")
        with open(snapshot_file_II, 'rb') as f:
            snapshots_II = pickle.load(f)
    else:
        print("Snapshot file not found, calculating snapshots.")
        snapshots = load_snapshots(filename, seed_indices_II)

        # Save the snapshot dict for future use
        with open(snapshot_file_II, 'wb') as f:
            pickle.dump(snapshots, f)
        print(f"Snapshots saved to {snapshot_file_II}")
    
    rot_angle = []
    print("Now running rotation calculation.")
    for (key_I, snap_data_I), (key_II, snap_data_II) in zip(snapshots_I.items(), snapshots_II.items()):
        rot = (compute_quaternion_rotation(snap_data_I, snap_data_II))
        print(2*np.arccos(rot[0]))
        rot_angle.append(rot)
if __name__ == '__main__':
    main()

Box Size: 181.7376018642
Loading snapshots from snapshots_I.pkl
Loading snapshots from snapshots_II.pkl
Now running rotation calculation.
3.1415926535897936
3.1415926535897913
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.1415926535897936
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.1415926535897936
3.1415926535897936
3.141592653589793
3.141592653589793
3.1415926535897936
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.1415926535897936
3.141592653589793
3.141592653589793
3.141592653589793
3.141592653589793
3.1415926535897936
3.141592653589793
3.141592653589794
3.141592